# frequency

> Functionality for frequency analysis.

In [ ]:
#| default_exp frequency

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import time
import polars as pl
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE

In [ ]:
#| export
class Frequency:
	""" Class for frequency analysis reporting """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| export
@patch
def frequencies(self: Frequency,
				normalize_by:int=1000000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return
				page_current:int=1, # current page
				show_token_id:bool=False, # show token_id in output
				exclude_punctuation:bool=True, # exclude punctuation tokens
				exclude_spaces:bool=True # exclude space tokens
				) -> Result: # return a Result object with the frequency table
	""" Report frequent tokens. """
	# TODO - allow choice of index ORTH, LOWER
	# TODO - add in restrict_to and exclude options - latter is for stopword removal
	
	start_time = time.time()
	self.corpus._init_frequency_table()

	columns = ['rank', 'token', 'frequency']
	if show_token_id == True:
		columns = ['rank', 'token_id', 'token', 'frequency']

	count_tokens = self.corpus.token_count
	tokens_descriptor = 'all tokens'
	if exclude_punctuation and exclude_spaces:
		count_tokens = self.corpus.word_token_count
		tokens_descriptor = 'word tokens'
	elif exclude_punctuation:
		count_tokens = self.corpus.word_token_count + len(self.corpus.space_tokens)
		tokens_descriptor = 'word and space tokens'
	elif exclude_spaces:
		count_tokens = self.corpus.word_token_count + len(self.corpus.punct_tokens)
		tokens_descriptor = 'word and punctuation tokens'

	formatted_data = []
	formatted_data.append(f'Report based on {tokens_descriptor}')

	# if a number is passed then normalize by that number
	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')
	self.corpus.frequency_table = self.corpus.frequency_table.with_columns((pl.col('frequency') * normalize_by / count_tokens).alias('normalized_frequency'))
	columns.append('normalized_frequency')

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')
	formatted_data.append(f'Total tokens: {count_tokens:,.0f}')

	logger.info(f'Frequencies report time: {(time.time() - start_time):.5f} seconds')

	df = self.corpus.frequency_table.sort('frequency', descending=True)
	if exclude_punctuation:
		df = df.filter(pl.col('is_punct') == False)
	if exclude_spaces:
		df = df.filter(pl.col('is_space') == False)
	unique_tokens = len(df)

	df = df.slice((page_current-1)*page_size, page_current*page_size)[columns]
	df = df.drop('rank').with_row_index(name='rank', offset=(page_current-1)*page_size+1)

	formatted_data.append(f'Unique tokens: {unique_tokens:,.0f}')
	formatted_data.append(f'Showing {page_size} rows')
	formatted_data.append(f'Page {page_current} of {count_tokens // page_size + 1}')

	return Result(type = 'frequencies', df=df, title='Frequencies', description='Frequencies of tokens in the corpus', summary_data={}, formatted_data=formatted_data)


In [ ]:
#| hide
path_to_corpus_file = '../test-corpora/saved/brown.corpus'

In [ ]:
# load the corpus
brown = Corpus().load(path_to_corpus_file)

In [ ]:
# instantiate the Frequency class
report_brown = Frequency(brown)

In [ ]:
# run the frequencies method and display the results
report_brown.frequencies(normalize_by=10000, page_size=20).display()

Frequencies 
 
 
 Frequencies of tokens in the corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 63516 
 647.56 
 
 
 2 
 of 
 36321 
 370.30 
 
 
 3 
 and 
 27787 
 283.30 
 
 
 4 
 to 
 25868 
 263.73 
 
 
 5 
 a 
 22190 
 226.23 
 
 
 6 
 in 
 19751 
 201.37 
 
 
 7 
 that 
 10409 
 106.12 
 
 
 8 
 is 
 10138 
 103.36 
 
 
 9 
 was 
 9931 
 101.25 
 
 
 10 
 for 
 8905 
 90.79 
 
 
 11 
 with 
 7043 
 71.81 
 
 
 12 
 it 
 6991 
 71.28 
 
 
 13 
 he 
 6772 
 69.04 
 
 
 14 
 as 
 6738 
 68.70 
 
 
 15 
 his 
 6523 
 66.50 
 
 
 16 
 on 
 6459 
 65.85 
 
 
 17 
 be 
 6365 
 64.89 
 
 
 18 
 's 
 5285 
 53.88 
 
 
 19 
 had 
 5200 
 53.02 
 
 
 20 
 by 
 5156 
 52.57 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total tokens: 980,844 
 


 
 Unique tokens: 42,907 
 


 
 Showing 20 rows 
 


 
 Page 1 of 49043

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()